In [98]:
import pandas as pd
import os
import re
from glob import glob

In [89]:
# Parameters
base_dir = r"D:\MSIM 4th sem\PR\2025Spring_Final_F1\Fastf1"  # You can change this path if needed
years = [2018, 2019, 2021, 2022, 2023, 2024]

# Helper function to extract driver number from filename
def extract_driver_number(filename):
    match = re.search(r'driver_(\d+)', filename)
    return match.group(1) if match else None

# Process each year
for year in years:
    # Load lap data for the year
    lap_file = os.path.join(base_dir, f"monaco_laps_{year}.csv")
    df_lap = pd.read_csv(lap_file)
    df_lap['Time'] = pd.to_timedelta(df_lap['Time'])

    # Prepare list to collect all car data for this year
    car_files = sorted(glob(os.path.join(base_dir, f"car_data_{year}_driver_*.csv")))
    car_data_frames = []

    for car_file in car_files:
        driver_number = extract_driver_number(car_file)
        df_car = pd.read_csv(car_file)
        df_car['Time'] = pd.to_timedelta(df_car['Time'])
        df_car['DriverNumber'] = int(driver_number)
        car_data_frames.append(df_car)

    # Combine all car data for the year
    df_car = pd.concat(car_data_frames, ignore_index=True)

    # Annotate df_car with LapNumber
    df_car['LapNumber'] = None

    # Preprocess for efficiency
    df_lap = df_lap[['DriverNumber', 'LapNumber', 'Time']].copy()
    df_lap['Time'] = pd.to_timedelta(df_lap['Time'])
    df_lap = df_lap.sort_values(['DriverNumber', 'LapNumber'])

    # Iterate through each driver's laps to assign LapNumber in df_car
    for driver in df_lap['DriverNumber'].unique():
        driver_laps = df_lap[df_lap['DriverNumber'] == driver].sort_values('LapNumber')
        driver_car_data = df_car['DriverNumber'] == driver

        for i in range(len(driver_laps) - 1):
            lap_start = driver_laps.iloc[i]['Time']
            lap_end = driver_laps.iloc[i + 1]['Time']
            lap_number = driver_laps.iloc[i]['LapNumber']

            in_lap = driver_car_data & (df_car['Time'] >= lap_start) & (df_car['Time'] < lap_end)
            df_car.loc[in_lap, 'LapNumber'] = lap_number

        # Assign last lap for each driver
        last_lap = driver_laps.iloc[-1]
        in_last_lap = driver_car_data & (df_car['Time'] >= last_lap['Time'])
        df_car.loc[in_last_lap, 'LapNumber'] = last_lap['LapNumber']
    df_car = df_car.dropna(subset=['LapNumber'])

    # Save the annotated file for this year
    df_car.to_csv(os.path.join(base_dir, f"car_data_with_lap_{year}.csv"), index=False)

In [116]:
years = [2018, 2019, 2021, 2022, 2023, 2024]
all_dfs = []

for year in years:
    print(f"Processing year {year}...")
    file_path = f"car_data_with_lap_{year}.csv"
    df = pd.read_csv(file_path)

    # Ensure correct dtypes
    df['Brake'] = df['Brake'].astype(bool)
    df['LapNumber'] = df['LapNumber'].astype(int)

    df = df[(df['nGear'] < 9)]

    # Aggregation
    agg_df = df.groupby(['DriverNumber', 'LapNumber']).agg({
        'RPM': 'mean',
        'nGear': 'mean',
        'Throttle': 'mean',
        'Brake': lambda x: x.sum(),  # True count
        'DRS': lambda x: x.isin([10, 12, 14]).sum()
    }).reset_index()

    #Saving as CSV
    output_file = f"aggregated_car_data_{year}.csv"
    agg_df.to_csv(output_file, index=False)

    try:
        df = pd.read_csv(output_file)
        df["Year"] = year  # optionally annotate year
        all_dfs.append(df)
    except FileNotFoundError:
        print(f"File for {year} not found. Skipping...")

df_all_years_car = pd.concat(all_dfs, ignore_index=True)

Processing year 2018...
Processing year 2019...
Processing year 2021...
Processing year 2022...
Processing year 2023...
Processing year 2024...


In [117]:
df_all_years_car

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year
0,2,1,9507.728070,3.751462,45.207602,98,0,2018
1,2,2,9467.687500,3.797619,44.747024,107,15,2018
2,2,3,9421.166667,3.827381,43.125000,101,20,2018
3,2,4,9429.387387,3.843844,42.303303,102,18,2018
4,2,5,9452.302395,3.787425,41.700599,108,18,2018
...,...,...,...,...,...,...,...,...
8306,81,74,9448.278912,3.693878,48.397959,84,0,2024
8307,81,75,9520.927835,3.680412,49.233677,80,0,2024
8308,81,76,9501.748322,3.684564,48.345638,83,0,2024
8309,81,77,9522.473498,3.692580,48.823322,79,0,2024


In [103]:
df_lap_2018 = pd.read_csv('monaco_laps_2018.csv')
df_lap_2019 = pd.read_csv('monaco_laps_2019.csv')
df_lap_2021 = pd.read_csv('monaco_laps_2021.csv')
df_lap_2022 = pd.read_csv('monaco_laps_2022.csv')
df_lap_2023 = pd.read_csv('monaco_laps_2023.csv')
df_lap_2024 = pd.read_csv('monaco_laps_2024.csv')

In [104]:
df_laps = pd.concat([df_lap_2018, df_lap_2019, df_lap_2021, df_lap_2022, df_lap_2023, df_lap_2024])

In [105]:
df_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year
0,0 days 00:08:19.100000,GAS,10,0 days 00:01:31.614000,1.0,NaN,NaN,NaN,NaN,0 days 00:00:39.994000,...,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False,2018
1,0 days 00:09:39.719000,GAS,10,0 days 00:01:20.619000,2.0,1.0,NaN,NaN,0 days 00:00:21.571000,0 days 00:00:37.371000,...,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True,2018
2,0 days 00:10:59.531000,GAS,10,0 days 00:01:19.812000,3.0,1.0,NaN,NaN,0 days 00:00:21.179000,0 days 00:00:37.273000,...,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True,2018
3,0 days 00:12:19.093000,GAS,10,0 days 00:01:19.562000,4.0,1.0,NaN,NaN,0 days 00:00:21.050000,0 days 00:00:37.245000,...,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True,2018
4,0 days 00:13:38.493000,GAS,10,0 days 00:01:19.400000,5.0,1.0,NaN,NaN,0 days 00:00:21.054000,0 days 00:00:37.039000,...,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,0 days 03:14:09.170000,ZHO,24,0 days 00:01:20.585000,72.0,3.0,NaN,NaN,0 days 00:00:20.370000,0 days 00:00:36.805000,...,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True,2024
1222,0 days 03:15:24.403000,ZHO,24,0 days 00:01:15.233000,73.0,3.0,NaN,NaN,0 days 00:00:19.940000,0 days 00:00:35.135000,...,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True,2024
1223,0 days 03:16:44.876000,ZHO,24,0 days 00:01:20.473000,74.0,3.0,NaN,NaN,0 days 00:00:21.051000,0 days 00:00:37.048000,...,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True,2024
1224,0 days 03:17:59.594000,ZHO,24,0 days 00:01:14.718000,75.0,3.0,NaN,NaN,0 days 00:00:19.751000,0 days 00:00:34.990000,...,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True,2024


In [118]:
merged_df = pd.merge(
    df_all_years_car,
    df_laps,
    on=['DriverNumber', 'LapNumber', 'Year'],
    how='right'
)

In [119]:
merged_df

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year,Time,Driver,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,10,1,9531.494048,3.413690,46.101190,99,0,2018,0 days 00:08:19.100000,GAS,...,True,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False
1,10,2,9394.374251,3.592814,47.080838,93,16,2018,0 days 00:09:39.719000,GAS,...,False,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True
2,10,3,9429.505952,3.583333,48.086310,89,24,2018,0 days 00:10:59.531000,GAS,...,False,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True
3,10,4,9437.838906,3.589666,47.723404,85,22,2018,0 days 00:12:19.093000,GAS,...,False,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True
4,10,5,9427.572727,3.600000,46.830303,92,19,2018,0 days 00:13:38.493000,GAS,...,False,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,24,72,9382.875887,3.932624,50.872340,90,0,2024,0 days 03:14:09.170000,ZHO,...,True,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True
8307,24,73,8971.025559,3.750799,43.853035,102,0,2024,0 days 03:15:24.403000,ZHO,...,True,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True
8308,24,74,9461.837545,4.014440,52.931408,85,0,2024,0 days 03:16:44.876000,ZHO,...,True,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True
8309,24,75,9103.670175,3.870175,47.568421,88,0,2024,0 days 03:17:59.594000,ZHO,...,True,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True


In [121]:
merged_df.columns

Index(['DriverNumber', 'LapNumber', 'RPM', 'nGear', 'Throttle', 'Brake', 'DRS',
       'Year', 'Time', 'Driver', 'LapTime', 'Stint', 'PitOutTime', 'PitInTime',
       'Sector1Time', 'Sector2Time', 'Sector3Time', 'Sector1SessionTime',
       'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife',
       'FreshTyre', 'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus',
       'Position', 'Deleted', 'DeletedReason', 'FastF1Generated',
       'IsAccurate'],
      dtype='object')

In [122]:
merged_df.to_csv('cars_laps_monaco_all_years.csv', index=False)